<a href="https://colab.research.google.com/github/samer-glitch/Federated-Governance-and-Provenance-Scoring-for-Trustworthy-AI-A-Metadata-Ledger-Approach/blob/main/Plan_A_CIFAR_10_TADP_vs_Centralized_Vs_FL_5_clients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- 0. Install/Import ---

!pip uninstall -y cryptography
!pip install --upgrade --force-reinstall cryptography
!pip install --upgrade --force-reinstall "flwr[simulation]==1.18.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.0/540.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you h

In [1]:

import os, glob, shutil, uuid, random, time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import flwr as fl

# --- 1. Configs & Constants ---

NUM_CLIENTS            = 5
NUM_ROUNDS_FL          = 5
DEFAULT_LOCAL_EPOCHS   = 1
DEFAULT_CLIENT_FRACTION = 1.0
RANDOM_STATE           = 42
POWER_W                = 45
COST_PER_KWH           = 0.2
CARBON_INTENSITY       = 0.475
version_counters = {}

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# --- 2. Load & Partition CIFAR-10 ---

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32') / 255.0
X_test  = X_test.astype('float32') / 255.0
y_train = y_train.flatten()
y_test  = y_test.flatten()

# Shuffle and partition into 5 clients using Dirichlet
idx = np.random.permutation(len(X_train))
X_train, y_train = X_train[idx], y_train[idx]
fracs = np.random.dirichlet([1.5]*NUM_CLIENTS)
client_data_splits = {}
start = 0
for i, frac in enumerate(fracs):
    end = start + int(frac*len(X_train))
    client_data_splits[str(i)] = (X_train[start:end], y_train[start:end])
    start = end
if start < len(X_train):
    last = str(NUM_CLIENTS-1)
    X_extra = np.concatenate([client_data_splits[last][0], X_train[start:]], axis=0)
    y_extra = np.concatenate([client_data_splits[last][1], y_train[start:]], axis=0)
    client_data_splits[last] = (X_extra, y_extra)

CLIENT_IDS = list(client_data_splits.keys())

# --- 3. Ledger Directory Setup ---

if os.path.exists("./ledgers"):
    shutil.rmtree("./ledgers")
os.makedirs("./ledgers/local", exist_ok=True)
os.makedirs("./ledgers/central", exist_ok=True)

pd.DataFrame(columns=[
    "tx_id","timestamp","client","version","record_count",
    "dim1","dim2","dim3","dim4","dim5","dim6",
    "pscore","action"
]).to_csv("./ledgers/central/central_ledger.csv", index=False)

# --- 4. TADP Quality Profiles (Image-adapted, can be improved) ---

quality_profiles = {
    'high':    {'dim1':5, 'dim2':5, 'dim3':5, 'dim4':5, 'dim5':5, 'dim6':5},
    'moderate':{'dim1':3, 'dim2':3, 'dim3':3, 'dim4':3, 'dim5':3, 'dim6':3},
    'low':     {'dim1':1, 'dim2':1, 'dim3':1, 'dim4':1, 'dim5':1, 'dim6':1},
}
def assign_pscore_templates(scenario):
    profiles = {}
    if scenario == 'high':
        for c in CLIENT_IDS: profiles[c] = quality_profiles['high']
    elif scenario == 'varied':
        for i, c in enumerate(CLIENT_IDS):
            lvl = 'high' if i == 0 else 'moderate' if i < NUM_CLIENTS-1 else 'low'
            profiles[c] = quality_profiles[lvl]
    else:
        hc = random.choice(CLIENT_IDS)
        for c in CLIENT_IDS:
            lvl = 'high' if c == hc else 'low'
            profiles[c] = quality_profiles[lvl]
    return profiles

WEIGHTS_PSCORE = {
    'dim1':0.25,'dim2':0.15,'dim3':0.10,'dim4':0.10,'dim5':0.30,'dim6':0.10
}
def policy_pscore(score):
    if   score >= 4.0: return 'Excellent','ACCEPT'
    elif score >= 3.2: return 'Good','ACCEPT'
    elif score >= 2.5: return 'Moderate','REVIEW'
    else:              return 'Poor','QUARANTINE'

def run_tadp_governance(client_data, client_profiles):
    central="./ledgers/central/central_ledger.csv"
    rows=[]
    for cid in CLIENT_IDS:
        p = client_profiles[cid]
        dim_vals = [p[f'dim{i+1}'] for i in range(6)]
        score = sum(WEIGHTS_PSCORE[f'dim{i+1}'] * dim_vals[i] for i in range(6))
        label,action = policy_pscore(score)
        rows.append({
            'client':cid,'pscore':round(score,2),'action':action,
            **{f'dim{i+1}':dim_vals[i] for i in range(6)}
        })
    gov_df = pd.DataFrame(rows).set_index('client')
    # Ledger write as in your original pipeline (optional, left for you to expand)
    return gov_df

# --- 5. Model Definition (CNN for CIFAR-10) ---

def make_model(input_shape=(32,32,3), num_classes=10):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax'),
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# --- 6. Flower Client for Images ---

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, X, y):
        self.model = make_model()
        self.X = X
        self.y = y

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.X, self.y, epochs=DEFAULT_LOCAL_EPOCHS, verbose=0, batch_size=64)
        return self.model.get_weights(), len(self.X), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.X, self.y, verbose=0)
        return loss, len(self.X), {"accuracy": acc}

def client_fn(cid):
    idx = int(cid)
    X, y = client_data_splits[str(idx)]
    return FlowerClient(X, y)

def start_fl_simulation(num_clients=NUM_CLIENTS, X_test=None, y_test=None):
    def evaluate(rnd, params, cfg):
        model = make_model()
        model.set_weights(params)
        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        return loss, {"accuracy": acc}
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=DEFAULT_CLIENT_FRACTION,
        min_fit_clients=num_clients,
        min_available_clients=num_clients,
        evaluate_fn=evaluate,
    )
    hist = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS_FL),
        strategy=strategy,
        client_resources={"num_cpus": 1},
    )
    return hist

# --- 7. Scenario Execution Loop (simplified: All Accepted, can expand as in your original code) ---

scenario_configs = {
    ("All Accepted", "Federated Full"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_federated": True,
        "is_tadp": False
    },
    ("All Accepted", "TADP Federated"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_federated": True,
        "is_tadp": True
    },
    # add more scenarios as needed
}

METRICS = ['accuracy']

results = {}
for scenario_name, config in scenario_configs.items():
    print(f"\n===== Scenario: {scenario_name} =====")
    gov_df = run_tadp_governance(client_data_splits, config["profile"])
    accepted = gov_df[gov_df['action']=='ACCEPT'].index.tolist()
    accepted_idx = [CLIENT_IDS.index(c) for c in accepted]
    accepted_splits = [client_data_splits[str(i)] for i in accepted_idx]

    # Federated training
    t0 = time.time()
    hist = start_fl_simulation(num_clients=len(accepted_splits), X_test=X_test, y_test=y_test)
    train_time = time.time() - t0

    last_acc = hist.metrics_centralized['accuracy'][-1][1] if ('accuracy' in hist.metrics_centralized) else np.nan
    results[scenario_name] = {
        'train_time': train_time,
        'test_accuracy': last_acc,
        'accepted_clients': accepted,
    }
    print(f"  FL Test Accuracy: {last_acc:.4f}  |  Time: {train_time:.1f} s")

# --- 8. Centralized Benchmark for Reference ---

model = make_model()
model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Centralized test accuracy: {acc:.4f}")

# --- Results Table ---

df_results = pd.DataFrame([
    {"Scenario": k, "Test Acc": v["test_accuracy"], "Clients": len(v["accepted_clients"]), "Time (s)": v["train_time"]}
    for k, v in results.items()
])
print("\n==== Scenario Results ====")
display(df_results)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout



===== Scenario: ('All Accepted', 'Federated Full') =====


2025-07-14 08:23:31,970	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7989537179.0, 'object_store_memory': 3994768588.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=3759) 2025-07-14 08:23:38.143858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=3759) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=3759) E0000 00:00:1752481418.264824    3759 cuda_dnn.cc:8310] U

  FL Test Accuracy: 0.6008  |  Time: 386.4 s

===== Scenario: ('All Accepted', 'TADP Federated') =====


2025-07-14 08:30:00,947	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 8003656091.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 4001828044.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=6616) 2025-07-14 08:30:06.601071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=6616) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=6616) E0000 00:00:1752481806.632525    6616 cuda_dnn.cc:8310] U

  FL Test Accuracy: 0.5938  |  Time: 374.6 s
Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step - accuracy: 0.3572 - loss: 1.7580
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 47ms/step - accuracy: 0.5540 - loss: 1.2635
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 46ms/step - accuracy: 0.6148 - loss: 1.1048
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 42s 47ms/step - accuracy: 0.6510 - loss: 1.0067
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 47ms/step - accuracy: 0.6772 - loss: 0.9383
Centralized test accuracy: 0.6613

==== Scenario Results ====


,Scenario,Test Acc,Clients,Time (s)
0,"(All Accepted, Federated Full)",0.6008,5,386.380889
1,"(All Accepted, TADP Federated)",0.5938,5,374.568498


In [5]:
# ────────────── 0. IMPORTS & CONFIG ──────────────
import os, io, glob, time, random, uuid, shutil, string, itertools
from datetime import datetime
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from IPython.display import display
import tensorflow as tf, flwr as fl
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

NUM_CLIENTS, RANDOM_STATE, NUM_ROUNDS_FL = 5, 42, 5
DEFAULT_LOCAL_EPOCHS, DEFAULT_CLIENT_FRACTION = 1, 1.0
CLIENT_IDS = (list(string.ascii_uppercase) + [''.join(p) for p in itertools.product(string.ascii_uppercase, repeat=2)])[:NUM_CLIENTS]
version_counters = {}
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
tf.keras.utils.set_random_seed(RANDOM_STATE)
tf.get_logger().setLevel("ERROR")
print(f"TensorFlow: {tf.__version__}, Flower: {fl.__version__}")

# ────────────── 1. LEDGER FOLDERS INIT ──────────────
if os.path.exists("./ledgers"):
    shutil.rmtree("./ledgers")
os.makedirs("./ledgers/local", exist_ok=True)
os.makedirs("./ledgers/central", exist_ok=True)
pd.DataFrame(columns=[
    "tx_id","timestamp","client","version","record_count",
    "dim1","dim2","dim3","dim4","dim5","dim6","pscore","action"
]).to_csv("./ledgers/central/central_ledger.csv", index=False)

# ────────────── 2. LOAD & SPLIT CIFAR-10 ──────────────
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train, y_test = y_train.flatten(), y_test.flatten()
X_train, X_test = X_train.astype('float32')/255., X_test.astype('float32')/255.

# Shuffle and split indices to clients
indices = np.random.permutation(len(X_train))
fracs = list(np.random.dirichlet([1.5]*NUM_CLIENTS))
client_data_splits, start = {}, 0
for cid in CLIENT_IDS:
    if not fracs: break
    frac = fracs.pop()
    end = start + int(frac*len(X_train))
    idx = indices[start:end]
    client_data_splits[cid] = (X_train[idx], y_train[idx])
    start = end
if start < len(X_train):
    cid = CLIENT_IDS[-1]
    idx = indices[start:]
    Xs, ys = client_data_splits[cid]
    client_data_splits[cid] = (np.concatenate([Xs, X_train[idx]]), np.concatenate([ys, y_train[idx]]))
print("\nClient Data Distribution:")
for cid, (Xc, yc) in client_data_splits.items():
    print(f" • {cid}: {len(Xc)} samples")

# ────────────── 3. MODEL & METRICS ──────────────
def make_model(input_shape=(32,32,3), num_classes=10, verbose=True):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

def get_detailed_metrics(y_true, y_proba):
    y_pred = np.argmax(y_proba, axis=1)
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'f1': f1_score(y_true, y_pred, average='weighted', zero_division=0),
    }

# ────────────── 4. TADP GOVERNANCE ──────────────
def random_quality_profile(size):
    # Simulate pscore profile by size/other synthetic feature, for demo only
    # You can replace with actual CIFAR-10 metadata if you have
    if size > 20000:   return 4.5    # Excellent
    elif size > 10000: return 3.6    # Good
    elif size > 5000:  return 2.7    # Review
    else:              return 1.9    # Quarantine

def policy_pscore(score):
    if   score>=4.0: return 'Excellent','ACCEPT'
    elif score>=3.2: return 'Good','ACCEPT'
    elif score>=2.5: return 'Moderate','REVIEW'
    else:            return 'Poor','QUARANTINE'

def run_tadp_governance(client_data):
    rows=[]
    for cid, (Xc, yc) in client_data.items():
        size = len(Xc)
        score = random_quality_profile(size)
        label, action = policy_pscore(score)
        rows.append({'client':cid,'pscore':round(score,2),'action':action})
    gov_df = pd.DataFrame(rows).set_index('client')
    display(gov_df)
    # Ledger writing (minimal, expand as needed)
    out = gov_df.reset_index()
    out['tx_id']        = [uuid.uuid4().hex for _ in range(len(out))]
    out['timestamp']    = datetime.utcnow().isoformat()
    out['record_count'] = out['client'].map(lambda c: len(client_data[c][0]))
    out['version']      = [version_counters.get(c,0)+1 for c in out['client']]
    for i, c in enumerate(out['client']):
        version_counters[c] = out.at[i, 'version']
    out['dim1']=out['dim2']=out['dim3']=out['dim4']=out['dim5']=out['dim6']=out['pscore']
    out = out[["tx_id","timestamp","client","version","record_count","dim1","dim2","dim3","dim4","dim5","dim6","pscore","action"]]
    out.to_csv("./ledgers/central/central_ledger.csv", mode='a', header=False, index=False)
    for _, r in out.iterrows():
        lp = f"./ledgers/local/{r.client}_ledger.csv"
        pd.DataFrame([r]).to_csv(lp, mode='a', header=not os.path.exists(lp), index=False)
    return gov_df

def record_review_decision(client_id, version, decision):
    ledger_path = f"./ledgers/local/{client_id}_ledger.csv"
    df = pd.read_csv(ledger_path)
    last_entries = df[df['version'] == version]
    if last_entries.empty:
        print(f"[Warning] No entry found for client {client_id} version {version} in ledger!")
        return
    last_entry = last_entries.iloc[-1]
    new_entry = last_entry.copy()
    new_entry['action'] = decision
    new_entry['timestamp'] = datetime.utcnow().isoformat()
    new_entry['tx_id'] = uuid.uuid4().hex
    df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
    df.to_csv(ledger_path, index=False)
    central_path = "./ledgers/central/central_ledger.csv"
    central_df = pd.read_csv(central_path)
    cols = central_df.columns.tolist()
    new_central = {col: new_entry[col] if col in new_entry else (last_entry[col] if col in last_entry else np.nan) for col in cols}
    central_df = pd.concat([central_df, pd.DataFrame([new_central])], ignore_index=True)
    central_df.to_csv(central_path, index=False)

# ────────────── 5. FLWR CLIENT & SERVER ──────────────
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, X, y):
        self.cid, self.model, self.X, self.y = cid, model, X, y
    def get_parameters(self, config): return self.model.get_weights()
    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.X, self.y, epochs=DEFAULT_LOCAL_EPOCHS, batch_size=64, verbose=0)
        return self.model.get_weights(), len(self.X), {}
    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.X, self.y, verbose=0)
        yhat = self.model.predict(self.X, verbose=0)
        metrics = get_detailed_metrics(self.y, yhat)
        return loss, len(self.X), metrics

def client_fn(cid, clients, client_data):
    client_id = clients[int(cid)]
    X, y = client_data[client_id]
    model = make_model(verbose=False)
    return FlowerClient(client_id, model, X, y).to_client()

def start_fl_simulation(clients, client_data, X_test, y_test):
    def evaluate(rnd, params, cfg):
        m = make_model(verbose=False)
        m.set_weights(params)
        p = m.predict(X_test, verbose=0)
        return 0, get_detailed_metrics(y_test, p)
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=DEFAULT_CLIENT_FRACTION,
        min_fit_clients=len(clients),
        min_available_clients=len(clients),
        evaluate_fn=evaluate
    )
    hist = fl.simulation.start_simulation(
        client_fn=lambda cid: client_fn(cid, clients, client_data),
        num_clients=len(clients),
        config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS_FL),
        strategy=strategy,
        client_resources={'num_cpus':1}
    )
    return hist

def get_fl_metrics(hist):
    if hasattr(hist, "metrics_centralized") and 'accuracy' in hist.metrics_centralized:
        return {'accuracy': float(hist.metrics_centralized['accuracy'][-1][1])}
    return {'accuracy': float('nan')}

# ────────────── 6. INTERACTIVE SCENARIO: TADP VARIED ──────────────
scenario_results = {}
print(f"\n===== Scenario: TADP Varied Review (CIFAR-10) =====")
gov_df = run_tadp_governance(client_data_splits)
accepted = gov_df[gov_df['action'] == 'ACCEPT'].index.tolist()
review_clients = gov_df[gov_df['action'] == 'REVIEW'].index.tolist()

for cid in review_clients:
    print(f"\n🟠 Client {cid} is in REVIEW. Showing sample images shapes: {client_data_splits[cid][0][:2].shape}")
    print(gov_df.loc[cid])
    ans = input(f"👉 Accept client {cid} for training? [y/n]: ").strip().lower()
    if ans == 'y':
        accepted.append(cid)
        record_review_decision(cid, version_counters.get(cid, 1), 'ACCEPT')
    else:
        record_review_decision(cid, version_counters.get(cid, 1), 'QUARANTINE')
accepted = sorted(list(set(accepted)))
accepted_data = {c: client_data_splits[c] for c in accepted}
Xc = np.concatenate([accepted_data[c][0] for c in accepted])
yc = np.concatenate([accepted_data[c][1] for c in accepted])

# Centralized
print("\n--- Centralized Training on Accepted Data ---")
t0 = time.time()
model = make_model(verbose=False)
model.fit(Xc, yc, epochs=3, batch_size=64, verbose=1)
train_time = time.time() - t0
metrics = get_detailed_metrics(y_test, model.predict(X_test))
scenario_results['centralized'] = dict(metrics=metrics, param_count=model.count_params(), train_time=train_time)

# Federated
print("\n--- Federated Training on Accepted Data ---")
t0 = time.time()
hist = start_fl_simulation(accepted, client_data_splits, X_test, y_test)
train_time = time.time() - t0
metrics = get_fl_metrics(hist)
scenario_results['federated'] = dict(metrics=metrics, train_time=train_time)

print("\n==== RESULTS SUMMARY ====")
for approach, res in scenario_results.items():
    print(f"\n{approach.capitalize()} Training:")
    for k, v in res.items():
        print(f"  {k}: {v}")


TensorFlow: 2.18.0, Flower: 1.18.0

Client Data Distribution:
 • A: 18531 samples
 • B: 9830 samples
 • C: 2247 samples
 • D: 9282 samples
 • E: 10110 samples

===== Scenario: TADP Varied Review (CIFAR-10) =====


,pscore,action
client,,
A,3.6,ACCEPT
B,2.7,REVIEW
C,1.9,QUARANTINE
D,2.7,REVIEW
E,3.6,ACCEPT



🟠 Client B is in REVIEW. Showing sample images shapes: (2, 32, 32, 3)
pscore       2.7
action    REVIEW
Name: B, dtype: object
👉 Accept client B for training? [y/n]: Y

🟠 Client D is in REVIEW. Showing sample images shapes: (2, 32, 32, 3)
pscore       2.7
action    REVIEW
Name: D, dtype: object
👉 Accept client D for training? [y/n]: Y

--- Centralized Training on Accepted Data ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff91c07f12c12850069839bc7901000000 Worker ID: 75567d6057d2df6d3bd4624dc4ddfc7fbcd4b192f10c833f52036bb9 Node ID: 23864855988b09fcdf140b645f94c02b61579150bd079051108f49e6 Worker IP address: 172.28.0.12 Worker port: 40713 Worker PID: 6616 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.
Epoch 1/3
662/747 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.3522 - loss: 1.7693

...

747/747 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.3635 - loss: 1.7409
Epoch 2/3
747/747 ━━━━━━━━━━━━━━━━━━━━ 41s 51ms/step - accuracy: 0.5788 - loss: 1.2009
Epoch 3/3
747/747 ━━━━━━━━━━━━━━━━━━━━ 38s 51ms/step - accuracy: 0.6335 - loss: 1.0549
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout



--- Federated Training on Accepted Data ---


2025-07-14 09:04:38,408	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7999296308.0, 'object_store_memory': 3999648153.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=19327) 2025-07-14 09:04:45.235860: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=19327) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=19327) E0000 00:00:1752483885.266472   19327 cuda_dnn.cc:8310


==== RESULTS SUMMARY ====

Centralized Training:
  metrics: {'accuracy': 0.6375, 'precision': 0.6636384011634741, 'recall': 0.6375, 'f1': 0.6420318080429819}
  param_count: 167562
  train_time: 125.63488864898682

Federated Training:
  metrics: {'accuracy': 0.6086}
  train_time: 429.3899812698364


**MODULAR DATA - CIFAR-10 USE-CASE:


1.   60000 IMAGES SPLIT INTO 50000 TRAINIGN AND 10000 TESTING PARTITIONED INTO K=5
2.   CLIENTS randomly using dirichtet method
3. CNN model: Conv2D(32) + MaxPool + Conv2D(64) + MaxPool + Flatten + Dense(64) + Output.
4. Centralized: model trained on all accepted data; Federated: Flower runs with clients.
5. TADP logic: as described, with automated batch decision for "REVIEW" (set to 'y' or 'n'). but datasets marked as REVIEW are being automatically accepted for training without administrator approval or prompting. JUST TO SPEED UP THE PROCESS.
5. All key metrics are computed for all 8 scenarios and displayed in a Q1-style summary.
**

In [2]:
# =================== Cell 0: SETUP ======================
# !pip install -q tensorflow flwr[simulation] matplotlib seaborn
!pip install tensorflow flwr ray matplotlib seaborn
import os, shutil, gc, time, uuid, random, string, itertools, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import flwr as fl
from datetime import datetime
from collections import OrderedDict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

warnings.filterwarnings("ignore")
tf.get_logger().setLevel("ERROR")

NUM_CLIENTS = 5
RANDOM_STATE = 42
NUM_ROUNDS_FL = 5
DEFAULT_LOCAL_EPOCHS = 2
DEFAULT_CLIENT_FRACTION = 1.0
BATCH_SIZE = 64

CARBON_INTENSITY = 0.475
COST_PER_KWH = 0.2
POWER_W = 45  # System power

# Unique client IDs
single = list(string.ascii_uppercase)
CLIENT_IDS = (single)[:NUM_CLIENTS]
version_counters = {}

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# =============== Cell 1: LEDGER SETUP ===================
if os.path.exists("./ledgers"):
    shutil.rmtree("./ledgers")
os.makedirs("./ledgers/local", exist_ok=True)
os.makedirs("./ledgers/central", exist_ok=True)

pd.DataFrame(columns=[
    "tx_id","timestamp","client","version","record_count",
    "dim1","dim2","dim3","dim4","dim5","dim6","pscore","action"
]).to_csv("./ledgers/central/central_ledger.csv", index=False)

# =========== Cell 2: CIFAR-10 LOADING & SPLIT ===========
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train, y_test = y_train.flatten(), y_test.flatten()

# Normalize
x_train, x_test = x_train/255.0, x_test/255.0

# Split into 5 clients, varied sizes (Dirichlet, like in your code)
def split_clients(X, y, n_clients=5, seed=42):
    np.random.seed(seed)
    total = len(X)
    fracs = np.random.dirichlet([1.5]*n_clients)
    fracs = np.round(fracs * total).astype(int)
    # Adjust to ensure sum = total
    diff = total - fracs.sum()
    fracs[0] += diff
    client_data = OrderedDict()
    idx = 0
    for i, cid in enumerate(CLIENT_IDS):
        end = idx + fracs[i]
        client_data[cid] = (X[idx:end], y[idx:end])
        idx = end
    return client_data

client_data_splits = split_clients(x_train, y_train, NUM_CLIENTS, RANDOM_STATE)
print("TensorFlow:", tf.__version__, "Flower:", fl.__version__)
print("\nClient Data Distribution:")
for cid, (x, y) in client_data_splits.items():
    print(f" • {cid}: {len(x)} samples")

# ========== Cell 3: CNN MODEL + METRICS ============
def make_cnn_model(input_shape=(32,32,3), n_classes=10, verbose=True):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(n_classes, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    if verbose:
        model.summary()
    return model

def get_metrics(y_true, y_proba):
    y_pred = np.argmax(y_proba, axis=1)
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'f1': f1_score(y_true, y_pred, average='weighted', zero_division=0)
    }

# ========== Cell 4: TADP GOVERNANCE FUNCTIONS ===========
WEIGHTS_PSCORE = {
    'dim1':0.25,'dim2':0.15,'dim3':0.10,'dim4':0.10,'dim5':0.30,'dim6':0.10
}
quality_profiles = {
    'high':    {'dim1':5, 'dim2':5, 'dim3':5, 'dim4':5, 'dim5':5, 'dim6':5},
    'moderate':{'dim1':3.5, 'dim2':3, 'dim3':3, 'dim4':3, 'dim5':3, 'dim6':3},
    'low':     {'dim1':2, 'dim2':2, 'dim3':2, 'dim4':2, 'dim5':2, 'dim6':2}
}
def assign_pscore_templates(scenario):
    profiles={}
    N=len(CLIENT_IDS)
    if scenario=='high':
        for c in CLIENT_IDS: profiles[c]=quality_profiles['high']
    elif scenario=='varied':
        t1,t2=int(N*0.2),int(N*0.7)
        for i,c in enumerate(CLIENT_IDS):
            lvl = 'high' if i<t1 else 'moderate' if i<t2 else 'low'
            profiles[c]=quality_profiles[lvl]
    else:  # low
        hc=random.choice(CLIENT_IDS)
        for c in CLIENT_IDS:
            lvl='high' if c==hc else 'low'
            profiles[c]=quality_profiles[lvl]
    return profiles

def policy_pscore(score):
    if   score>=4.0: return 'Excellent','ACCEPT'
    elif score>=3.2: return 'Good','ACCEPT'
    elif score>=2.5: return 'Moderate','REVIEW'
    else:            return 'Poor','QUARANTINE'

def run_tadp_governance(client_data, client_profiles):
    central="./ledgers/central/central_ledger.csv"
    rows=[]
    for cid, (X, y) in client_data.items():
        dims = [client_profiles[cid][f'dim{i+1}'] for i in range(6)]
        score = sum(WEIGHTS_PSCORE[f'dim{i+1}']*dims[i] for i in range(6))
        label,action = policy_pscore(score)
        rows.append({'client':cid,'pscore':round(score,2),'action':action, **{f'dim{i+1}':round(dims[i],2) for i in range(6)}})
    gov_df = pd.DataFrame(rows).set_index('client')
    display(gov_df)
    # Ledger appends
    out=gov_df.reset_index()
    out['tx_id']        = [uuid.uuid4().hex for _ in range(len(out))]
    out['timestamp']    = datetime.utcnow().isoformat()
    out['record_count'] = out['client'].map(lambda c: len(client_data[c][0]))
    out['version']      = 0
    for i,c in enumerate(out['client']):
        version_counters[c]=version_counters.get(c,0)+1
        out.at[i,'version']=version_counters[c]
    write=out[["tx_id","timestamp","client","version","record_count","dim1","dim2","dim3","dim4","dim5","dim6","pscore","action"]]
    write.to_csv(central, mode='a', header=False, index=False)
    for _,r in write.iterrows():
        lp=f"./ledgers/local/{r.client}_ledger.csv"
        pd.DataFrame([r]).to_csv(lp, mode='a', header=not os.path.exists(lp), index=False)
    return gov_df

def record_review_decision(client_id, version, decision, ledger_dir="./ledgers/local/"):
    ledger_path = f"{ledger_dir}/{client_id}_ledger.csv"
    df = pd.read_csv(ledger_path)
    last_entries = df[df['version'] == version]
    if last_entries.empty: return
    last_entry = last_entries.iloc[-1]
    new_entry = last_entry.copy()
    new_entry['action'] = decision
    new_entry['timestamp'] = datetime.utcnow().isoformat()
    new_entry['tx_id'] = uuid.uuid4().hex
    df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
    df.to_csv(ledger_path, index=False)
    central_path = "./ledgers/central/central_ledger.csv"
    central_df = pd.read_csv(central_path)
    cols = central_df.columns.tolist()
    new_central = {col: new_entry[col] if col in new_entry else (last_entry[col] if col in last_entry else np.nan) for col in cols}
    central_df = pd.concat([central_df, pd.DataFrame([new_central])], ignore_index=True)
    central_df.to_csv(central_path, index=False)

# ========== Cell 5: FEDERATED CLIENTS ===============
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, x, y):
        self.cid   = cid
        self.model = model
        self.x     = x
        self.y     = y

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x, self.y, epochs=DEFAULT_LOCAL_EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        return self.model.get_weights(), len(self.x), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x, self.y, verbose=0)
        yhat = self.model.predict(self.x, verbose=0)
        metrics = get_metrics(self.y, yhat)
        return loss, len(self.x), metrics

def client_fn(cid, clients, client_data):
    client_id = clients[int(cid)]
    x, y = client_data[client_id]
    model = make_cnn_model(input_shape=(32,32,3), n_classes=10, verbose=False)
    return FlowerClient(client_id, model, x, y).to_client()

def start_fl_simulation(clients, client_data, input_shape, x_test, y_test):
    def evaluate(rnd, params, cfg):
        m = make_cnn_model(input_shape, 10, verbose=False)
        m.set_weights(params)
        p = m.predict(x_test, verbose=0)
        metrics = get_metrics(y_test, p)
        return 0, metrics

    strategy = fl.server.strategy.FedAvg(
        fraction_fit=DEFAULT_CLIENT_FRACTION,
        min_fit_clients=len(clients),
        min_available_clients=len(clients),
        evaluate_fn=evaluate
    )

    hist = fl.simulation.start_simulation(
        client_fn=lambda cid: client_fn(cid, clients, client_data),
        num_clients=len(clients),
        config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS_FL),
        strategy=strategy,
        client_resources={'num_cpus':1}
    )
    return hist

def get_fl_metrics(hist, metrics_list):
    results = {}
    if hasattr(hist, "metrics_centralized"):
        for m in metrics_list:
            vals = hist.metrics_centralized.get(m, [])
            if isinstance(vals, list) and len(vals) > 0:
                results[m] = float(vals[-1][1])
            else:
                results[m] = float('nan')
    return results

# ======= Cell 6: 8 SCENARIOS AUTOMATED (BATCH-MODE) =======
scenario_configs = {
    ("All Accepted", "Centralized Full"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_centralized": True,
        "run_federated": False,
        "is_tadp": False
    },
    ("All Accepted", "TADP Centralized"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_centralized": True,
        "run_federated": False,
        "is_tadp": True
    },
    ("All Accepted", "Federated Full"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_centralized": False,
        "run_federated": True,
        "is_tadp": False
    },
    ("All Accepted", "TADP Federated"): {
        "profile": assign_pscore_templates('high'),
        "accept_all": True,
        "run_centralized": False,
        "run_federated": True,
        "is_tadp": True
    },
    ("Varied Review", "TADP Centralized"): {
        "profile": assign_pscore_templates('varied'),
        "accept_all": False,
        "run_centralized": True,
        "run_federated": False,
        "is_tadp": True
    },
    ("Varied Review", "TADP Federated"): {
        "profile": assign_pscore_templates('varied'),
        "accept_all": False,
        "run_centralized": False,
        "run_federated": True,
        "is_tadp": True
    },
    ("Single Dataset Accepted", "TADP Centralized"): {
        "profile": assign_pscore_templates('low'),
        "accept_all": False,
        "run_centralized": True,
        "run_federated": False,
        "is_tadp": True
    },
    ("Single Dataset Accepted", "TADP Federated"): {
        "profile": assign_pscore_templates('low'),
        "accept_all": False,
        "run_centralized": False,
        "run_federated": True,
        "is_tadp": True
    }
}

METRICS = ['accuracy', 'precision', 'recall', 'f1']
results = {}

for scenario_name, config in scenario_configs.items():
    print(f"\n===== Scenario: {scenario_name[0]} - {scenario_name[1]} (CIFAR-10) =====")
    gc.collect()
    tf.keras.backend.clear_session()

    if config["is_tadp"] or True:
        gov_df = run_tadp_governance(client_data_splits, config["profile"])
        if config["accept_all"]:
            accepted = CLIENT_IDS.copy()
        else:
            accepted = gov_df[gov_df['action'] == 'ACCEPT'].index.tolist()
            review_clients = gov_df[gov_df['action'] == 'REVIEW'].index.tolist()
            # For batch: auto-accept all "REVIEW" clients (set to 'n' to reject all)
            for cid in review_clients:
                version = version_counters.get(cid, 1)
                ans = 'y'  # For camera-ready, auto-accept all REVIEWs
                if ans == 'y':
                    accepted.append(cid)
                    record_review_decision(cid, version, 'ACCEPT')
                else:
                    record_review_decision(cid, version, 'QUARANTINE')
    else:
        accepted = CLIENT_IDS.copy()
        gov_df = None

    # Prepare training data for this scenario
    accepted_data = {c: client_data_splits[c] for c in accepted}
    # Aggregate all accepted client data for centralized
    if config["run_centralized"]:
        x_train_scen = np.concatenate([accepted_data[c][0] for c in accepted])
        y_train_scen = np.concatenate([accepted_data[c][1] for c in accepted])
        x_test_scen, y_test_scen = x_test, y_test
        t0 = time.time()
        model = make_cnn_model(input_shape=(32,32,3), n_classes=10, verbose=False)
        model.fit(x_train_scen, y_train_scen, epochs=3, batch_size=BATCH_SIZE, verbose=0)
        train_time = time.time() - t0
        metrics = get_metrics(y_test_scen, model.predict(x_test_scen, verbose=0))
        scenario_results = {
            'param_count': model.count_params(),
            'train_time': train_time,
            'metrics': metrics,
            'model_type': 'centralized',
            'is_tadp': config["is_tadp"]
        }
        del model
    elif config["run_federated"]:
        t0 = time.time()
        hist = start_fl_simulation(accepted, client_data_splits, (32,32,3), x_test, y_test)
        train_time = time.time() - t0
        metrics = get_fl_metrics(hist, METRICS)
        scenario_results = {
            'param_count': make_cnn_model((32,32,3), 10, False).count_params(),
            'train_time': train_time,
            'metrics': metrics,
            'model_type': 'federated',
            'is_tadp': config["is_tadp"]
        }
        del hist
    else:
        continue
    results[scenario_name] = {**scenario_results, 'accepted_clients': accepted, 'gov_df': gov_df}

# ======= Cell 7: Q1 SUMMARY TABLE OF ALL 8 SCENARIOS ========
def calculate_communication_cost(params, rounds):
    return params * 4 * 2 * rounds / (1024**2)
def calculate_energy_cost(time_s, devices=1):
    joules = POWER_W * time_s * devices
    kwh = joules / 3.6e6
    cost = kwh * COST_PER_KWH
    co2 = kwh * CARBON_INTENSITY
    return kwh, cost, co2

performance_data = []
for scenario_name, res in results.items():
    scenario, approach = scenario_name
    metrics = {
        'Scenario': scenario,
        'Approach': approach,
        **res['metrics'],
        'Parameters': res['param_count'],
        'Clients': len(res['accepted_clients']),
        'Time_s': res['train_time']
    }
    if 'Centralized' in approach:
        kwh, cost, co2 = calculate_energy_cost(res['train_time'])
        metrics.update({
            'Comm_MB': 0,
            'Energy_kWh': kwh,
            'Cost_USD': cost,
            'CO2_kg': co2,
            'Rounds': 1
        })
    else:
        comm = calculate_communication_cost(res['param_count'], NUM_ROUNDS_FL)
        kwh, cost, co2 = calculate_energy_cost(res['train_time'], len(res['accepted_clients']))
        metrics.update({
            'Comm_MB': comm,
            'Energy_kWh': kwh,
            'Cost_USD': cost,
            'CO2_kg': co2,
            'Rounds': NUM_ROUNDS_FL
        })
    performance_data.append(metrics)
column_order = [
    'Scenario', 'Approach', 'accuracy', 'precision', 'recall', 'f1',
    'Parameters', 'Clients', 'Time_s', 'Comm_MB', 'Energy_kWh', 'Cost_USD', 'CO2_kg', 'Rounds'
]
analysis_df = pd.DataFrame(performance_data)
analysis_df = analysis_df[column_order]
print("\n📊 Performance Analysis for All 8 Scenarios (CIFAR-10):")
display(analysis_df)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.1/598.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
TensorFlow: 2.19.0 Flower: 1.19.0

Client Data Distribution:
 • A: 10509 samples
 • B: 6010 samples
 • C: 5470 samples
 • D: 5470 samples
 • E: 22541 samples

===== Scenario: All Accepted - Centralized Full (CIFAR-10) =====


,pscore,action,dim1,dim2,dim3,dim4,dim5,dim6
client,,,,,,,,
A,5.0,ACCEPT,5,5,5,5,5,5
B,5.0,ACCEPT,5,5,5,5,5,5
C,5.0,ACCEPT,5,5,5,5,5,5
D,5.0,ACCEPT,5,5,5,5,5,5
E,5.0,ACCEPT,5,5,5,5,5,5



===== Scenario: All Accepted - TADP Centralized (CIFAR-10) =====


,pscore,action,dim1,dim2,dim3,dim4,dim5,dim6
client,,,,,,,,
A,5.0,ACCEPT,5,5,5,5,5,5
B,5.0,ACCEPT,5,5,5,5,5,5
C,5.0,ACCEPT,5,5,5,5,5,5
D,5.0,ACCEPT,5,5,5,5,5,5
E,5.0,ACCEPT,5,5,5,5,5,5


...


===== Scenario: Single Dataset Accepted - TADP Federated (CIFAR-10) =====


,pscore,action,dim1,dim2,dim3,dim4,dim5,dim6
client,,,,,,,,
A,5.0,ACCEPT,5,5,5,5,5,5
B,2.0,QUARANTINE,2,2,2,2,2,2
C,2.0,QUARANTINE,2,2,2,2,2,2
D,2.0,QUARANTINE,2,2,2,2,2,2
E,2.0,QUARANTINE,2,2,2,2,2,2


Setting `min_available_clients` lower than `min_fit_clients` or
`min_evaluate_clients` can cause the server to fail when there are too few clients
connected to the server. `min_available_clients` must be set to a value larger
than or equal to the values of `min_fit_clients` and `min_evaluate_clients`.

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2025-07-14 10:11:37,443	WARNING utils.py:592 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary work


📊 Performance Analysis for All 8 Scenarios (CIFAR-10):


,Scenario,Approach,accuracy,precision,recall,f1,Parameters,Clients,Time_s,Comm_MB,Energy_kWh,Cost_USD,CO2_kg,Rounds
0,All Accepted,Centralized Full,0.6303,0.633574,0.6303,0.622764,167562,5,147.450216,0.000000,0.001843,0.000369,0.000875,1
1,All Accepted,TADP Centralized,0.6454,0.643456,0.6454,0.639037,167562,5,145.471738,0.000000,0.001818,0.000364,0.000864,1
2,All Accepted,Federated Full,0.6714,0.678066,0.6714,0.672336,167562,5,776.996436,6.391983,0.048562,0.009712,0.023067,5
3,All Accepted,TADP Federated,0.6652,0.661106,0.6652,0.660857,167562,5,783.833959,6.391983,0.048990,0.009798,0.023270,5
4,Varied Review,TADP Centralized,0.5698,0.585018,0.5698,0.569501,167562,3,66.585536,0.000000,0.000832,0.000166,0.000395,1
5,Varied Review,TADP Federated,0.6096,0.629642,0.6096,0.612697,167562,3,400.760509,6.391983,0.015029,0.003006,0.007139,5
6,Single Dataset Accepted,TADP Centralized,0.5751,0.587711,0.5751,0.574001,167562,1,68.268006,0.000000,0.000853,0.000171,0.000405,1
7,Single Dataset Accepted,TADP Federated,0.6027,0.602961,0.6027,0.594942,167562,1,162.705840,6.391983,0.002034,0.000407,0.000966,5


In [11]:
# Columns you want in Table 1 (add 'Rounds' at the end)
perf_cols = ['Scenario', 'Approach', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'Time_s', 'Rounds']
present_perf_cols = [col for col in perf_cols if col in analysis_df.columns]

perf_table = analysis_df[present_perf_cols]

perf_styled = perf_table.style
for col, cmap in [
    ('accuracy', color_maps['performance']['accuracy']),
    ('precision', color_maps['performance']['precision']),
    ('recall', color_maps['performance']['recall']),
    ('f1', color_maps['performance']['f1']),
    ('roc_auc', color_maps['performance']['roc_auc']),
    ('Time_s', color_maps['performance']['Time_s'])
]:
    if col in perf_table.columns:
        perf_styled = perf_styled.background_gradient(cmap=cmap, subset=[col])

# Add formatters
perf_formatters = {}
for col, fmt in [
    ('accuracy', '{:.2%}'),
    ('precision', '{:.2%}'),
    ('recall', '{:.2%}'),
    ('f1', '{:.2%}'),
    ('roc_auc', '{:.2%}'),
    ('Time_s', '{:.1f} s'),
    ('Rounds', '{:.0f}')
]:
    if col in perf_table.columns:
        perf_formatters[col] = fmt

perf_styled = perf_styled.format(perf_formatters).set_caption('Table 1: Model Performance Metrics by Scenario')
display(perf_styled)
# Columns you want in Table 2
resource_cols = ['Scenario', 'Approach', 'Comm_MB', 'Energy_kWh', 'Cost_USD', 'CO2_kg', 'Rounds']
present_resource_cols = [col for col in resource_cols if col in analysis_df.columns]

# Prepare table
resource_table = analysis_df[present_resource_cols]

# Style the table: only apply gradient if column exists
resource_styled = resource_table.style

for col, cmap in [
    ('Comm_MB', color_maps['resources']['Comm_MB']),
    ('Energy_kWh', color_maps['resources']['Energy_kWh']),
    ('Cost_USD', color_maps['resources']['Cost_USD']),
    ('CO2_kg', color_maps['resources']['CO2_kg'])
]:
    if col in resource_table.columns:
        resource_styled = resource_styled.background_gradient(cmap=cmap, subset=[col])

# Format only columns that exist
resource_formatters = {}
for col, fmt in [
    ('Comm_MB', '{:.1f} MB'),
    ('Energy_kWh', '{:.3f} kWh'),
    ('Cost_USD', '${:.3f}'),
    ('CO2_kg', '{:.3f} kg'),
    ('Rounds', '{:.0f}')
]:
    if col in resource_table.columns:
        resource_formatters[col] = fmt

resource_styled = resource_styled.format(resource_formatters).set_caption('Table 2: Resource Usage and Costs by Scenario')
display(resource_styled)


,Scenario,Approach,accuracy,precision,recall,f1,Time_s,Rounds
0,All Accepted,Centralized Full,63.03%,63.36%,63.03%,62.28%,147.5 s,1
1,All Accepted,TADP Centralized,64.54%,64.35%,64.54%,63.90%,145.5 s,1
2,All Accepted,Federated Full,67.14%,67.81%,67.14%,67.23%,777.0 s,5
3,All Accepted,TADP Federated,66.52%,66.11%,66.52%,66.09%,783.8 s,5
4,Varied Review,TADP Centralized,56.98%,58.50%,56.98%,56.95%,66.6 s,1
5,Varied Review,TADP Federated,60.96%,62.96%,60.96%,61.27%,400.8 s,5
6,Single Dataset Accepted,TADP Centralized,57.51%,58.77%,57.51%,57.40%,68.3 s,1
7,Single Dataset Accepted,TADP Federated,60.27%,60.30%,60.27%,59.49%,162.7 s,5


,Scenario,Approach,Comm_MB,Energy_kWh,Cost_USD,CO2_kg,Rounds
0,All Accepted,Centralized Full,0.0 MB,0.002 kWh,$0.000,0.001 kg,1
1,All Accepted,TADP Centralized,0.0 MB,0.002 kWh,$0.000,0.001 kg,1
2,All Accepted,Federated Full,6.4 MB,0.049 kWh,$0.010,0.023 kg,5
3,All Accepted,TADP Federated,6.4 MB,0.049 kWh,$0.010,0.023 kg,5
4,Varied Review,TADP Centralized,0.0 MB,0.001 kWh,$0.000,0.000 kg,1
5,Varied Review,TADP Federated,6.4 MB,0.015 kWh,$0.003,0.007 kg,5
6,Single Dataset Accepted,TADP Centralized,0.0 MB,0.001 kWh,$0.000,0.000 kg,1
7,Single Dataset Accepted,TADP Federated,6.4 MB,0.002 kWh,$0.000,0.001 kg,5
